In [1]:
from datasets import load_dataset
import pickle

In [2]:
dataset = load_dataset("wikipedia", "20220301.en", beam_runner="DirectRunner")

Found cached dataset wikipedia (C:/Users/k2002/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset = dataset["train"]

In [4]:
new_dataset = []
my_words = ["fruit", "fruits", "vegetable", "vegetables", "meat"]
for site in dataset:
    tmp_text = site["text"].split(" ")
    for word in my_words:
        if word in tmp_text:
            new_dataset.append(site)
            break

In [5]:
print(len(new_dataset))

77605


In [6]:
dataset = new_dataset

In [13]:
new_dataset = []
for site in dataset:
    tmp_text = site["text"].split(" ")
    if "eat" in tmp_text or "food" in tmp_text:
        new_dataset.append(site)

In [14]:
print(len(new_dataset))

21052


In [15]:
dataset = new_dataset

In [24]:
print(dataset[1456]["title"])

Olmecs


In [25]:
with open('dataset/my_dataset.bin', 'wb') as file:
    pickle.dump(dataset, file)

In [26]:
with open('dataset/my_dataset.bin', 'rb') as file:
    dataset = pickle.load(file)